# import and fonction definition

In [1]:
import cv2
import napari
from tifffile import imread
import tifffile
import pandas as pd
from tqdm import tqdm
import matplotlib.path as mplPath
from pathlib import Path
from scipy import ndimage as ndi
import json
from skimage.measure import find_contours

import rasterio
from rasterio.features import rasterize
import numpy as np


from shapely.geometry import Polygon
import geopandas as gpd

In [2]:


def get_gene_random_vector(list_gene, nb_pc):
    """
    Return a random gene2vect dictionary.
    Args:
        list_gene: name of gene index.
        nb_pc: size of the random vector.
    Returns:
        Dictionary mapping gene names to random vectors.
    """
    gene2vect = {
        gene: np.random.uniform(0, 1, nb_pc) 
        for gene in list_gene
    }
    return gene2vect


def from_shape_layer_to_mask(viewer,
                             layer_shape_name = "Shapes [1]",
                             image_shape = (400, 400)
                             ):
    assert layer_shape_name in [layer._name for layer in viewer.layers.__dict__["_list"]], f"layer {layer_shape_name} not found"
    final_mask = np.zeros(image_shape, dtype=np.uint16)
    for layer in  viewer.layers.__dict__["_list"]:
        if layer._name==layer_shape_name:
            # Step 1: Retrieve the polygon coordinates
            polygon = layer.data.copy()
            polygon = polygon.copy()
            # Step 2: Create a meshgrid
            y, x = np.mgrid[:image_shape[0], :image_shape[1]]
            # Step 3: Create a path object from the polygon coordinates
            for index_cell in range(len(polygon)):
                if polygon[index_cell].shape[-1] == 3:
                    polygon[index_cell] = polygon[index_cell][:, 1:]
                    print("3D shape detected, only the two last columns are kept 3D annotation are not supported yet")
                poly_path = mplPath.Path(polygon[index_cell])
                # Step 4: Create a boolean mask
                mask = poly_path.contains_points(np.vstack((x.flatten(), y.flatten())).T).reshape(x.shape)
                # Step 5: Convert the boolean mask to an integer mask
                mask = mask.astype(np.uint16).T
                final_mask[mask>0] = index_cell+1
    return final_mask, polygon

def generate_img_of_rna_spots(df_crop,
                              radius = 3,
                              gene_column = "gene",
                              gene2color = None,
                              image_shape = (256, 256, 3),
                              gaussian_kernel_size  : int = None,
                              mode = "color",
                              column_x  = "global_x",
                              column_y = "global_y"):
    """
    generate a RGB image of the spots
    Args:
        df_crop: dataframe with the spots
        image_shape: shape of the image
        radius: radius of the spots
        gene_column: name of the column containing the gene
        gene2color: dictionary with the color / vector of each gene
        image_shape : shape of the image
        gaussian_kernel_size : size of the gaussian kernel to apply to the image
        mode  :  choose in ["color, 'vector'] choose color to add rgb spot, vector otherwise to add the vector of the gene
    Returns:
    """


    ## set a random color for each gene :
    gene_list = list(df_crop[gene_column].unique())
    assert df_crop[column_x].max() <= image_shape[0] , "df has to be already scaled or image shape too small"
    assert df_crop[column_y].max() <= image_shape[1], "df has to be already scaled or image shape too small"
    ## todo  change the name gene2color to gene2vector

    if mode == "color":
        #raise NotImplementedError("color mode deprecated only use for annotation")
        ## todo  remove this part
        if gene2color is None:
            print(' set color to random')
            gene2color = get_gene_random_vector(gene_list, nb_pc=image_shape[-1])
        img = np.zeros(image_shape, dtype=np.uint8)

        for row in tqdm(df_crop.iterrows()):
            x = row[1][column_x]
            y = row[1][column_y]
            gene = row[1]["gene"]
            color = gene2color[gene]
            color = (int(color[0] * 255), int(color[1] * 255), int(color[2] * 255))
            img = cv2.circle(img, (int(x), int(y)), radius, color, -1)

    elif mode == "vector":
        img = np.zeros(image_shape, dtype=np.float32)

        for row in tqdm(df_crop.iterrows()):
            x = row[1][column_x] ## pixel have to be alrday scaled
            y = row[1][column_y]
            gene = row[1]["gene"]
            if isinstance(gene2color[gene], (int, float)):
                assert  image_shape[-1] ==1,  "gene vector should have the same size  as the image"
            else :
                assert  len(gene2color[gene]) == image_shape[-1], "gene vector should have the same size  as the image"

            if isinstance(gene2color[gene], (int, float)):
                vector = gene2color[gene]
            else:
                vector = gene2color[gene][:image_shape[-1]]
            radius = float(radius)
            if radius < 1:
                x_list_coord = [x]
                y_list_coord = [y]
                img[int(x), int(y), :] = vector
            else:
                x_list_coord = [np.max([0.0, x-radius]), np.min([x+radius, image_shape[0]])]
                y_list_coord = [np.max([0.0, y-radius]), np.min([y+radius, image_shape[1]])]
                img[int(y_list_coord[0]):int(y_list_coord[1]), int(x_list_coord[0]):int(x_list_coord[1]), :] = vector
    else:
        raise ValueError("mode should be in ['color', 'vector']")

    #### sparse images


    if gaussian_kernel_size is not None or gaussian_kernel_size != 0:
        for i in range(img.shape[-1]):
            img[:, :, i] = ndi.gaussian_filter(img[:, :, i], sigma=gaussian_kernel_size)


    return img



# set local Path 

In [3]:
anotator_name = 'tom'
path_cache_folder = Path("/home/tom/share/st/open_vizgen/RNAseg_DATASET/breast/entire_dataset_z3_cb1_dapi.zarr/.sopa_cache/rna_seg_1200_50/")
experiment_folder = "annotation_test_1907"
patch_index = 3828#4901

gene2color = np.load("/home/tom/share/st/open_vizgen/HumanBreastCancerPatient1/crop4000v2/gene_color/breast_v2/gene2color_npc5.npy",
                     allow_pickle=True).item()
gene2color_dist = np.load("/home/tom/share/st/open_vizgen/HumanBreastCancerPatient1/crop4000v2/gene_color/gene2vect_dist_rgb_min_max.npy",
                          allow_pickle=True).item()


# PREPARE ANNOTION IMAGES

In [4]:

## load the images
dapi = imread(path_cache_folder / str(patch_index) / experiment_folder / "dapi.tif")
cyto1 = imread(path_cache_folder / str(patch_index) / experiment_folder / "0_img_cellbound.tif")
cyto2 = imread(path_cache_folder / str(patch_index) / experiment_folder / "1_img_cellbound.tif")
cyto3 = imread(path_cache_folder / str(patch_index) / experiment_folder / "2_img_cellbound.tif")
try:
    label = imread(path_cache_folder / str(patch_index) / experiment_folder / "label.tif")
except FileNotFoundError:
    label = None

df_spots = pd.read_csv(path_cache_folder / str(patch_index) / "transcripts.csv")

# read json
with open(path_cache_folder / str(patch_index) / "bounds.json") as f:
    bounds = json.load(f)

## rescale the spots
df_spots["x"] = df_spots["x"] - bounds["bounds_min_x"]
df_spots["y"] = df_spots["y"] - bounds["bounds_min_y"]





custom_rna_plot_random  = generate_img_of_rna_spots(df_spots,
                                                    radius = 1,
                                                    gene_column = "gene",
                                                    gene2color = None,
                                                    image_shape = (1200, 1200, 3),
                                                    gaussian_kernel_size = 0,
                                                    mode = "color",
                                                    column_x  = "x",
                                                    column_y = "y")


custom_rna_plot_coexpression  = generate_img_of_rna_spots(
    df_crop=df_spots,
    radius = 1,
    gene_column = "gene",
    gene2color = gene2color,
    image_shape = (1200, 1200, 3),
    gaussian_kernel_size = 0,
    mode = "color",
    column_x  = "x",
    column_y = "y")




custom_rna_plot_dist_nuclei  = generate_img_of_rna_spots(
    df_crop = df_spots,
    radius = 1,
    gene_column = "gene",
    gene2color = gene2color_dist,
    image_shape = (1200, 1200, 3),
    gaussian_kernel_size = 0,
    mode = "color",
    column_x  = "x",
    column_y = "y")


 set color to random


30638it [00:01, 18040.74it/s]
30638it [00:01, 18226.70it/s]
30638it [00:01, 17845.65it/s]


# CREATE NAPARI VIEWER 

In [ ]:
    viewer = napari.Viewer()
    viewer.add_image(dapi, name='dapi')
    viewer.add_image(cyto1, name='cyto1')
    viewer.add_image(cyto2, name='cyto2')
    viewer.add_image(cyto3, name='cyto3')
    viewer.add_image(custom_rna_plot_random, name='rna')
    viewer.add_image(custom_rna_plot_coexpression, name='custom_rna_plot_coexpression')
    viewer.add_image(custom_rna_plot_dist_nuclei, name='custom_rna_plot_dist_nuclei')
    if label is not None:
        viewer.add_image(label, name='label')
        contours = find_contours(label, level=0.5)
        #viewer.add_shapes(contours, shape_type='polygon', edge_color='red', name='Segmentation')
    try:
        annotation = tifffile.imread(path_cache_folder / str(patch_index) / experiment_folder / f"{anotator_name}_annotation.tif")
        viewer.add_image(annotation, name='annotation')
        #contours = find_contours(annotation, level=0.5)
        #viewer.add_shapes(contours, shape_type='polygon', edge_color='red', name='Annotation')
    except:
        pass

Cannot move to target thread (0x559e5a851cf0)



#### Try to load annotation from other annotation

In [ ]:
ext_anotator_name = 'tom'
try:
        annotation = tifffile.imread(path_cache_folder / str(patch_index) / experiment_folder / f"{ext_anotator_name}_annotation.tif")
    viewer.add_image(annotation, name='annotation')
    #contours = find_contours(annotation, level=0.5)
    #viewer.add_shapes(contours, shape_type='polygon', edge_color='red', name='Annotation')
except:
    pass

# VIZUALIZE YOUR ANNOTATION FOR A LAST CHECK

In [ ]:
final_mask, polygon = from_shape_layer_to_mask(
    viewer,
    layer_shape_name = "Shapes",
    image_shape = dapi.shape,
)
print(f" {len(polygon)} polygons found")




viewer.add_image(final_mask, name='final_mask')


# SAVE YOUR  ANNOTATION 

In [ ]:
list_coord_poly = []
for coord_poly in polygon:
    list_coord_poly.append(coord_poly.copy())
## transpose the polygons
list_coord_poly_T = []
for coord_poly in list_coord_poly:
    list_coord_poly_T.append(coord_poly[:, [1, 0]].copy())


## shift polygone coordiante to global

for index_cell in range(len(polygon)):
    list_coord_poly_T[index_cell][:, 0] = list_coord_poly_T[index_cell][:, 0] + bounds["bounds_min_x"]
    list_coord_poly_T[index_cell][:, 1] = list_coord_poly_T[index_cell][:, 1] + bounds["bounds_min_y"]

list_poly = [Polygon(coord) for coord in list_coord_poly_T]
gdf_polygon = gpd.GeoDataFrame(geometry=list_poly)
print(gdf_polygon.geometry[0].bounds)


# save gdf and mask
gdf_polygon.to_file(str(path_cache_folder / str(patch_index) / experiment_folder / f"{anotator_name}_annotation.shp"))
tifffile.imwrite(path_cache_folder / str(patch_index) / experiment_folder / f"{anotator_name}_annotation.tif", final_mask)
annotation_tif = tifffile.imread(path_cache_folder / str(patch_index) / experiment_folder / f"{anotator_name}_annotation.tif")

## read .shp
gdf_polygon = gpd.read_file(path_cache_folder / str(patch_index) / experiment_folder / f"{anotator_name}_annotation.shp")